In [ ]:
!pip install -U fsspec==2023.6.0
!pip install peft
!pip install hf_xet
!pip install datasets
import os
import sys
import traceback
import torch
from transformers import RobertaModel
from torch.utils.data import DataLoader,TensorDataset
from transformers import RobertaTokenizer, DataCollatorWithPadding
from datasets import load_dataset
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from torch.cuda.amp import autocast, GradScaler
import torch.optim as optim
import torch.nn.functional as F

from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import FancyBboxPatch, ConnectionPatch
from peft import LoraConfig, get_peft_model, TaskType
import torch
import torch.nn as nn
from transformers import RobertaModel
from peft import LoraConfig, get_peft_model, TaskType


#os.environ["CUDA_VISIBLE_DEVICES"] = "2"

print(f"Python {sys.version}")

from matplotlib import pyplot as plt

"""
We need to make this determinitsic so we can keep a track of changes we do to the model
If we are using the same initialisation all the time, then changes
"""
import numpy as np
import pandas as pd

np.random.seed(317)

import random

random.seed(317)

try:
    import torch as pt

    DEVICE = 'cuda' if pt.cuda.is_available() else 'cpu'
    # DEVICE = 'cpu'
    print(f"PyTorch {pt.__version__}")
    print(f"DEVICE={DEVICE}")
    if pt.cuda.is_available():
        print(f"\tGPU: {pt.cuda.get_device_name(0)}")
        print(f"\t\tcapability: {pt.cuda.get_device_capability('cuda')[0]}")
        print(f"\tCUDA version: {pt.version.cuda}")
        print("\tcuDNN available: ", pt.backends.cudnn.is_available())

        if pt.backends.cudnn.is_available():
            print("\t\tcuDNN version: ", pt.backends.cudnn.version())

        # Print the number of GPUs available
        print(f"\tNumber of GPUs available: {pt.cuda.device_count()}")

    from torch import nn
    from torch.nn import functional as F
    from torch.optim import AdamW
    from torch.utils.data import Dataset, DataLoader

    pt.manual_seed(317)
except:
    print("No PyTorch")
    print(traceback.format_exc())

try:
    import tensorflow as tf

    print(f"TensorFlow {tf.__version__}")
    print(f"Build with CUDA: {tf.test.is_built_with_cuda()}")
    print(f"Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
except:
    print("No TensorFlow")

try:
  from transformers import BertTokenizer, BertForSequenceClassification
  from tqdm import tqdm

  #print(f"Tokenizer version {BertTokenizer.__version__}")
except:
  print("No Transformers")

try:
  import sklearn
  from sklearn.model_selection import train_test_split
  from sklearn.metrics import accuracy_score
  print(f"scikit-learn {sklearn.__version__}")
except:
  print("No scikit-learn")

# DEVICE="cpu"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.3 MB/s eta 0:00:00
Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
PyTorch 2.5.1+cu124
DEVICE=cuda
	GPU: NVIDIA A100-SXM4-40GB
		capability: 8
	CUDA version: 12.4
	cuDNN available:  True
		cuDNN version:  90100
	Number of GPUs available: 1
TensorFlow 2.18.0
Build with CUDA: True
Num GPUs Available:  1
scikit-learn 1.6.1


In [ ]:
dataset = load_dataset("snli")

def filter_valid_labels(example):
    return example['label'] != -1

dataset = dataset.filter(filter_valid_labels)

train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/412k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/413k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/19.6M [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/550152 [00:00<?, ? examples/s]

In [ ]:
train_df = pd.DataFrame(train_dataset)
test_df = pd.DataFrame(test_dataset)
val_df = pd.DataFrame(val_dataset)
pd.concat([train_df, test_df, val_df])["label"].nunique()

3

In [ ]:
class NLIClassifier(nn.Module):
    def __init__(self, hidden_dim=768, num_labels=3, num_layers=6, max_len=128, freeze_roberta=True):
        super().__init__()


        self.roberta = RobertaModel.from_pretrained('roberta-base')
        if freeze_roberta:
            for param in self.roberta.parameters():
                param.requires_grad = False

        lora_config = LoraConfig(
            task_type=TaskType.FEATURE_EXTRACTION,
            inference_mode=False,
            r=16,
            lora_alpha=32,
            lora_dropout=0.1,
            target_modules=[
                "query",
                "key",
                "value",
                "dense",
            ],
            bias="none",
        )

        self.roberta = get_peft_model(self.roberta, lora_config)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=8,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, num_labels)
        )

    def forward(self, input_ids, attention_mask):

        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        x = outputs.last_hidden_state
        x = self.transformer_encoder(x, src_key_padding_mask=~attention_mask.bool())
        cls_output = x[:, 0, :]

        return self.classifier(cls_output)

    def print_trainable_parameters(self):

        trainable_params = 0
        all_param = 0

        for _, param in self.named_parameters():
            all_param += param.numel()
            if param.requires_grad:
                trainable_params += param.numel()

        print(f"Trainable params: {trainable_params:,} || "
              f"All params: {all_param:,} || "
              f"Trainable%: {100 * trainable_params / all_param:.2f}%")

In [ ]:
class SimpleCrossAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads=8, dropout=0.1):
        super().__init__()
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )
        self.layer_norm = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key_value, attention_mask=None):
        attended_output, _ = self.cross_attention(
            query=query,
            key=key_value,
            value=key_value,
            key_padding_mask=attention_mask
        )

        output = self.layer_norm(query + self.dropout(attended_output))
        return output

In [ ]:
class NLIClassifier(nn.Module):
    def __init__(self, hidden_dim=768, num_labels=3, num_layers=6, max_len=128, freeze_roberta=True):
        super().__init__()

        self.roberta = RobertaModel.from_pretrained('roberta-base')
        if freeze_roberta:
            for param in self.roberta.parameters():
                param.requires_grad = False

        lora_config = LoraConfig(
            task_type=TaskType.FEATURE_EXTRACTION,
            inference_mode=False,
            r=16,
            lora_alpha=32,
            lora_dropout=0.1,
            target_modules=[
                "query",
                "key",
                "value",
                "dense",
            ],
            bias="none",
        )
        self.roberta = get_peft_model(self.roberta, lora_config)

        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=8, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.cross_attention = SimpleCrossAttention(hidden_dim, num_heads=8, dropout=0.1)

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, num_labels)
        )

    def _split_premise_hypothesis(self, input_ids, attention_mask):

        batch_size, seq_len = input_ids.size()
        sep_token_id = self.roberta.config.sep_token_id

        premise_masks = torch.zeros_like(attention_mask, dtype=torch.float, device=input_ids.device)
        hypothesis_masks = torch.zeros_like(attention_mask, dtype=torch.float, device=input_ids.device)


        for i in range(batch_size):
            valid_positions = attention_mask[i] == 1
            sep_positions = (input_ids[i] == sep_token_id) & valid_positions
            sep_indices = torch.where(sep_positions)[0]

            if len(sep_indices) > 0:
                sep_pos = sep_indices[0].item()


                if sep_pos > 1:
                    premise_masks[i, 1:sep_pos] = attention_mask[i, 1:sep_pos].float()


                if sep_pos + 1 < seq_len:
                    hypothesis_masks[i, sep_pos+1:] = attention_mask[i, sep_pos+1:].float()

            else:

                mid_point = seq_len // 2
                premise_masks[i, 1:mid_point] = attention_mask[i, 1:mid_point].float()
                hypothesis_masks[i, mid_point:] = attention_mask[i, mid_point:].float()


        for i in range(batch_size):
            if premise_masks[i].sum() == 0:
                premise_masks[i, 1] = 1.0
            if hypothesis_masks[i].sum() == 0:
                hypothesis_masks[i, -1] = 1.0

        return premise_masks, hypothesis_masks

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        x = outputs.last_hidden_state

        x = self.transformer_encoder(x, src_key_padding_mask=~attention_mask.bool())


        premise_masks, hypothesis_masks = self._split_premise_hypothesis(input_ids, attention_mask)


        premise_repr = x * premise_masks.unsqueeze(-1)
        hypothesis_repr = x * hypothesis_masks.unsqueeze(-1)


        enhanced_hypothesis = self.cross_attention(
            query=hypothesis_repr,
            key_value=premise_repr,
            attention_mask=~premise_masks.bool()
        )


        x = premise_repr + enhanced_hypothesis
        cls_output = x[:, 0, :]

        return self.classifier(cls_output)

    def print_trainable_parameters(self):
        trainable_params = 0
        all_param = 0
        for _, param in self.named_parameters():
            all_param += param.numel()
            if param.requires_grad:
                trainable_params += param.numel()

        print(f"Trainable params: {trainable_params:,} || "
              f"All params: {all_param:,} || "
              f"Trainable%: {100 * trainable_params / all_param:.2f}%")

In [ ]:
# class NLIClassifier(nn.Module):
#     def __init__(self,
#                  hidden_dim=768,
#                  num_labels=3,
#                  num_layers=3,
#                  num_cross_attn_layers=2,
#                  max_len=512,
#                  freeze_roberta=True,
#                  dropout=0.2):
#         super().__init__()

#         self.roberta = RobertaModel.from_pretrained('roberta-base')

#         if freeze_roberta:
#             for param in self.roberta.parameters():
#                 param.requires_grad = False
#             # for param in self.roberta.encoder.layer[-3:].parameters():
#             #     param.requires_grad = True

#         encoder_layer = nn.TransformerEncoderLayer(
#             d_model=hidden_dim,
#             nhead=8,
#             dropout=dropout,
#             batch_first=True,
#             activation='gelu'
#         )
#         self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

#         # self.cross_attention_layers = nn.ModuleList([
#         #     CrossAttentionLayer(hidden_dim, dropout=dropout)
#         #     for _ in range(num_cross_attn_layers)
#         # ])

#         self.cross_attention_layers = nn.ModuleList()

#         # self.attention_pooling = nn.MultiheadAttention(
#         #     embed_dim=hidden_dim,
#         #     num_heads=1,
#         #     batch_first=True
#         # )

#         # self.pooling_query = nn.Parameter(torch.empty(1, 1, hidden_dim))
#         # nn.init.kaiming_uniform_(self.pooling_query)

#         self.classifier = nn.Sequential(
#             nn.Dropout(0.1),
#             nn.Linear(hidden_dim*3, 128),
#             nn.ReLU(),
#             nn.Linear(128, 3)
#         )

#         self.dropout = nn.Dropout(dropout)


#     # def _init_weights(self):
#     #     for module in self.modules():
#     #         if isinstance(module, nn.Linear):
#     #             nn.init.kaiming_uniform_(module.weight)
#     #             if module.bias is not None:
#     #                 nn.init.zeros_(module.bias)
#     #         elif isinstance(module, nn.LayerNorm):
#     #             nn.init.ones_(module.weight)
#     #             nn.init.zeros_(module.bias)

#     def _split_premise_hypothesis(self, input_ids, attention_mask):
#         batch_size, seq_len = input_ids.size()

#         # Find [SEP] token positions
#         sep_token_id = self.roberta.config.sep_token_id

#         # Create masks for premise and hypothesis
#         premise_masks = torch.zeros_like(attention_mask, dtype=torch.float, device=input_ids.device)
#         hypothesis_masks = torch.zeros_like(attention_mask, dtype=torch.float, device=input_ids.device)

#         # Process each batch item
#         for i in range(batch_size):
#             valid_positions = attention_mask[i] == 1
#             sep_positions = (input_ids[i] == sep_token_id) & valid_positions
#             sep_indices = torch.where(sep_positions)[0]

#             if len(sep_indices) > 0:
#                 sep_pos = sep_indices[0].item()

#                 # Premise: from start (skip CLS) to first [SEP] (exclusive)
#                 if sep_pos > 1:  # Ensure we have some premise tokens
#                     premise_masks[i, 1:sep_pos] = attention_mask[i, 1:sep_pos].float()

#                 # Hypothesis: from first [SEP] (exclusive) to end
#                 if sep_pos + 1 < seq_len:  # Ensure we have some hypothesis tokens
#                     hypothesis_masks[i, sep_pos+1:] = attention_mask[i, sep_pos+1:].float()

#             else:
#                 # Fallback: split in half if no SEP token found
#                 mid_point = seq_len // 2
#                 premise_masks[i, 1:mid_point] = attention_mask[i, 1:mid_point].float()
#                 hypothesis_masks[i, mid_point:] = attention_mask[i, mid_point:].float()


#         # Ensure at least one token per mask
#         for i in range(batch_size):
#             if premise_masks[i].sum() == 0:
#                 premise_masks[i, 1] = 1.0  # At least one token
#             if hypothesis_masks[i].sum() == 0:
#                 hypothesis_masks[i, -1] = 1.0  # At least one token

#         return premise_masks, hypothesis_masks
#     def forward(self, input_ids, attention_mask):
#         # Get RoBERTa embeddings
#         outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
#         x = outputs.last_hidden_state

#         # Apply transformer encoder
#         src_key_padding_mask = ~attention_mask.bool()
#         x = self.transformer_encoder(x, src_key_padding_mask=src_key_padding_mask)

#         # SKIP ALL PREMISE/HYPOTHESIS SPLITTING
#         combined_repr = x  # Use representations directly, no splitting!

#         # Simple pooling
#         # 1. [CLS] token
#         cls_output = combined_repr[:, 0, :]

#         # 2. Max pooling
#         masked_repr = combined_repr * attention_mask.unsqueeze(-1)
#         masked_repr = masked_repr.masked_fill(~attention_mask.unsqueeze(-1).bool(), -1e9)
#         max_pooled = torch.max(masked_repr, dim=1)[0]

#         # 3. Mean pooling
#         masked_repr_mean = combined_repr * attention_mask.unsqueeze(-1)
#         mean_pooled = masked_repr_mean.sum(dim=1) / attention_mask.sum(dim=1, keepdim=True)

#         # Concatenate
#         final_repr = torch.cat([cls_output, max_pooled, mean_pooled], dim=-1)

#         # Classify
#         final_repr = self.dropout(final_repr)
#         logits = self.classifier(final_repr)

#         return logits
#     # def forward(self, input_ids, attention_mask):
#     #     # Get RoBERTa embeddings
#     #     outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
#     #     x = outputs.last_hidden_state  # [batch_size, seq_len, hidden_dim]

#     #     # Apply transformer encoder
#     #     src_key_padding_mask = ~attention_mask.bool()
#     #     x = self.transformer_encoder(x, src_key_padding_mask=src_key_padding_mask)

#     #     # Split premise and hypothesis
#     #     premise_masks, hypothesis_masks = self._split_premise_hypothesis(input_ids, attention_mask)

#     #     # Extract premise and hypothesis representations
#     #     premise_repr = x * premise_masks.unsqueeze(-1)
#     #     hypothesis_repr = x * hypothesis_masks.unsqueeze(-1)

#     #     # Apply simplified cross attention layers
#     #     enhanced_premise = premise_repr
#     #     enhanced_hypothesis = hypothesis_repr

#     #     for cross_attn_layer in self.cross_attention_layers:
#     #         # Premise attends to hypothesis
#     #         if hypothesis_masks.sum() > 0:
#     #             enhanced_premise = cross_attn_layer(
#     #                 query=enhanced_premise,
#     #                 key_value=enhanced_hypothesis,
#     #                 attention_mask=~hypothesis_masks.bool()
#     #             )

#     #         # Hypothesis attends to premise
#     #         if premise_masks.sum() > 0:
#     #             enhanced_hypothesis = cross_attn_layer(
#     #                 query=enhanced_hypothesis,
#     #                 key_value=enhanced_premise,
#     #                 attention_mask=~premise_masks.bool()
#     #             )

#     #     # Combine representations
#     #     combined_repr = enhanced_premise + enhanced_hypothesis

#     #     # Multiple pooling strategies
#     #     # 1. [CLS] token representation
#     #     cls_output = combined_repr[:, 0, :]

#     #     # 2. Max pooling with proper masking
#     #     masked_repr = combined_repr * attention_mask.unsqueeze(-1)
#     #     masked_repr = masked_repr.masked_fill(~attention_mask.unsqueeze(-1).bool(), -1e9)
#     #     max_pooled = torch.max(masked_repr, dim=1)[0]

#     #     # 3. Attention pooling (OLD - COMMENTED OUT)
#     #     # batch_size = combined_repr.size(0)
#     #     # pooling_query = self.pooling_query.expand(batch_size, -1, -1)
#     #     # attention_pooled, _ = self.attention_pooling(
#     #     #     query=pooling_query,
#     #     #     key=combined_repr,
#     #     #     value=combined_repr,
#     #     #     key_padding_mask=~attention_mask.bool(),
#     #     #     need_weights=False
#     #     # )
#     #     # attention_pooled = attention_pooled.squeeze(1)

#     #     # 3. Simple Mean pooling (NEW - REPLACES ATTENTION POOLING)
#     #     masked_repr_mean = combined_repr * attention_mask.unsqueeze(-1)
#     #     attention_pooled = masked_repr_mean.sum(dim=1) / attention_mask.sum(dim=1, keepdim=True)

#     #     # Concatenate all representations
#     #     final_repr = torch.cat([cls_output, max_pooled, attention_pooled], dim=-1)

#     #     # Apply dropout and classify
#     #     final_repr = self.dropout(final_repr)
#     #     logits = self.classifier(final_repr)

#     #     return logits

#     def print_trainable_parameters(self):
#         trainable_params = 0
#         all_param = 0
#         for _, param in self.named_parameters():
#             all_param += param.numel()
#             if param.requires_grad:
#                 trainable_params += param.numel()

#         print(f"Trainable params: {trainable_params:,} || "
#               f"All params: {all_param:,} || "
#               f"Trainable%: {100 * trainable_params / all_param:.2f}%")

In [ ]:
class MyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.premise = texts["premise"]
        self.hypothesis = texts["hypothesis"]
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.premise)

    def __getitem__(self, idx):
        premise = str(self.premise[idx])
        hypothesis = str(self.hypothesis[idx])
        label = self.labels[idx]


        encoding = self.tokenizer(
            premise,
            hypothesis,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': pt.tensor(label, dtype=pt.long)
        }

In [ ]:
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_loss = None
        self.counter = 0
        self.best_weights = None

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(model)
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self.save_checkpoint(model)
        else:
            self.counter += 1

        if self.counter >= self.patience:
            if self.restore_best_weights:
                model.load_state_dict(self.best_weights)
            return True
        return False

    def save_checkpoint(self, model):
        self.best_weights = model.state_dict().copy()


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
# train_texts = train_df[["premise", "hypothesis"]]
# train_labels = train_df["label"]

# val_texts = val_df[["premise", "hypothesis"]]
# val_labels = val_df["label"]

# test_texts = test_df[["premise", "hypothesis"]]
# test_labels = test_df["label"]

# train_dataset = MyDataset(train_texts, train_labels, tokenizer)
# val_dataset = MyDataset(val_texts, val_labels, tokenizer)
# test_dataset = MyDataset(test_texts, test_labels, tokenizer)

# train_loader = DataLoader(
#     train_dataset,
#     batch_size=32,
#     shuffle=True,
#     collate_fn=data_collator,
#     pin_memory=True,
#     drop_last=True,
# )

# val_loader = DataLoader(
#     val_dataset,
#     batch_size=32,
#     shuffle=False,
#     collate_fn=data_collator,
#     pin_memory=True,
# )

# test_loader = DataLoader(
#     test_dataset,  # Fixed the indexing error
#     batch_size=32,
#     shuffle=False,
#     collate_fn=data_collator,
#     pin_memory=True,
# )

In [ ]:
# def preprocess(df):
#     return tokenizer(
#         df["premise"],
#         df["hypothesis"],
#         truncation=True,
#         padding="max_length",
#         add_special_tokens=True,
#         max_length=128,
#         return_tensors='pt'
#     )

# encoded_dataset = dataset.map(preprocess)
# encoded_dataset = encoded_dataset.remove_columns(['premise', 'hypothesis'])


Map:   0%|          | 0/9824 [00:00<?, ? examples/s]

Map:   0%|          | 0/549367 [00:00<?, ? examples/s]

Map:   0%|          | 0/9842 [00:00<?, ? examples/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# save_path = '/content/drive/MyDrive/nli_encoded_dataset'
# encoded_dataset.save_to_disk(save_path)

Mounted at /content/drive


In [ ]:
def load_preprocessed_dataset():
    from datasets import load_from_disk

    save_path = '/content/drive/MyDrive/nli_encoded_dataset'

    if os.path.exists(save_path):
        print("Loading preprocessed dataset from Drive...")
        encoded_dataset = load_from_disk(save_path)
        print("Dataset loaded!")
        return encoded_dataset
    else:
        print("Preprocessed dataset not found. Run preprocessing first.")
        return None

encoded_dataset = load_preprocessed_dataset()

Loading preprocessed dataset from Drive...
Dataset loaded!


In [ ]:
# encoded_dataset = encoded_dataset.rename_column('label', 'labels')
encoded_dataset.set_format(columns=["input_ids", "attention_mask", "labels"])

In [ ]:
train_dataset = encoded_dataset["train"]
val_dataset  = encoded_dataset["validation"]
test_dataset = encoded_dataset["test"]

train_loader = DataLoader(
    train_dataset,
    batch_size=256,
    shuffle=True,
    collate_fn=data_collator,
    num_workers=12,
    pin_memory=True,
    drop_last=True,
    persistent_workers=True,
    prefetch_factor=2
)

val_loader = DataLoader(
    val_dataset,
    batch_size=256,
    shuffle=False,
    collate_fn=data_collator,
    num_workers=12,
    pin_memory=True,
    persistent_workers=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=256,
    shuffle=False,
    collate_fn=data_collator,
    num_workers=12,
    pin_memory=True,
    persistent_workers=True
)

In [ ]:
def train_epoch(model, dataloader, optimizer, criterion, device, scaler):
    model.train()
    total_loss = 0
    preds, targets = [], []
    bar = tqdm(dataloader, desc="Training", leave=False)

    for batch in bar:
        input_ids = batch['input_ids'].squeeze().to(device)
        attention_mask = batch['attention_mask'].squeeze().to(device)
        labels = batch['labels'].long().to(device)

        optimizer.zero_grad()

        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)


        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        predictions = torch.argmax(outputs, dim=-1).detach().cpu()
        preds.extend(predictions.numpy())
        targets.extend(labels.detach().cpu().numpy())

        bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(dataloader)
    acc = accuracy_score(targets, preds)
    return avg_loss, acc

In [ ]:
def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    preds, targets = [], []
    bar = tqdm(dataloader, desc="Test Inference", leave=False)
    with torch.no_grad():
        for batch in bar:
            input_ids = batch['input_ids'].squeeze().to(device)
            attention_mask = batch['attention_mask'].squeeze().to(device)
            labels = batch['labels'].long().to(device)

            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                outputs = model(input_ids, attention_mask)
                loss = criterion(outputs, labels)

            total_loss += loss.item()
            predictions = torch.argmax(outputs, dim=-1).detach().cpu()
            preds.extend(predictions.numpy())
            targets.extend(labels.detach().cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    acc = accuracy_score(targets, preds)
    return avg_loss, acc

In [ ]:
def predict_on_test_set(model, dataloader, device, label_map=None):
    model.eval()
    preds, targets = [], []
    bar = tqdm(dataloader, desc="Test Inference", leave=False)

    with torch.no_grad():
        for batch in bar:
            input_ids = batch['input_ids'].squeeze().to(device)
            attention_mask = batch['attention_mask'].squeeze().to(device)
            labels = batch['labels'].long().to(device)

            with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
                outputs = model(input_ids,attention_mask)

            predictions = torch.argmax(outputs, dim=-1).detach().cpu()
            preds.extend(predictions.numpy())
            targets.extend(labels.detach().cpu().numpy())


    if label_map:
        preds = [label_map[p] for p in preds]
        targets = [label_map[t] for t in targets]

    return targets, preds


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NLIClassifier(freeze_roberta=True).to(device)

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cuda.enable_flash_sdp(True)

scaler = GradScaler()
model = torch.compile(model, mode="max-autotune-no-cudagraphs")

optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=5e-4,
    weight_decay=1e-2,
    eps=1e-6,
    betas=(0.9, 0.95)
)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.2,
        patience=3
)

early_stopping = EarlyStopping(patience=3, min_delta=0.001, restore_best_weights=True)
criterion = nn.CrossEntropyLoss()

print("\n")
model.print_trainable_parameters()
print("\n")

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-19-ab5360ada9f2>:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()




Trainable params: 38,719,491 || All params: 163,365,123 || Trainable%: 23.70%




In [ ]:
# def debug_parameters(model):
#     print("=== Detailed Parameter Breakdown ===")

#     roberta_trainable = 0
#     roberta_frozen = 0
#     lora_params = 0
#     transformer_params = 0
#     classifier_params = 0

#     for name, param in model.named_parameters():
#         param_count = param.numel()

#         if 'roberta' in name.lower():
#             if param.requires_grad:
#                 roberta_trainable += param_count
#                 if 'lora' in name.lower():
#                     lora_params += param_count
#                     print(f"✅ LoRA: {name}")
#                 else:
#                     print(f"🚨 UNFROZEN RoBERTa: {name}")
#             else:
#                 roberta_frozen += param_count
#         elif 'transformer_encoder' in name.lower():
#             if param.requires_grad:
#                 transformer_params += param_count
#         elif 'classifier' in name.lower():
#             if param.requires_grad:
#                 classifier_params += param_count

#     print(f"\n=== Summary ===")
#     print(f"RoBERTa frozen: {roberta_frozen:,}")
#     print(f"RoBERTa trainable (LoRA): {roberta_trainable:,}")
#     print(f"  ├─ LoRA params: {lora_params:,}")
#     print(f"  └─ Other RoBERTa: {roberta_trainable - lora_params:,}")
#     print(f"Custom transformer: {transformer_params:,}")
#     print(f"Classifier: {classifier_params:,}")
#     print(f"TOTAL TRAINABLE: {roberta_trainable + transformer_params + classifier_params:,}")

# # Run this on your model:
# debug_parameters(model)

In [ ]:
history = {
    "train_loss": [],
    "train_acc": [],
    "val_loss": [],
    "val_acc": []
}

for epoch in range(1, 16):
    print(f"\nEpoch {epoch}")

    if torch.cuda.is_available():
      torch.cuda.empty_cache()

    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, device,scaler)
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    scheduler.step(val_loss)

    if early_stopping(val_loss, model):
        print(f"\nEarly stopping triggered after {epoch} epochs!")
        print("Restored best model weights.")
        break

    if val_loss == early_stopping.best_loss:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
        }, 'best_model_checkpoint.pt')


    history["train_loss"].append(train_loss)
    history["train_acc"].append(train_acc)
    history["val_loss"].append(val_loss)
    history["val_acc"].append(val_acc)

    current_lr = optimizer.param_groups[0]['lr']
    print(f"LR: {current_lr:.2e} | Best Val Loss: {early_stopping.best_loss:.4f}")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.4f}")



Epoch 1


Training:   0%|          | 0/2145 [00:00<?, ?it/s]AUTOTUNE mm(32768x16, 16x3072)
  triton_mm_251 0.1403 ms 100.0% ACC_TYPE='tl.float32', ALLOW_TF32=True, BLOCK_K=16, BLOCK_M=128, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=True, GROUP_M=8, num_stages=4, num_warps=8
  triton_mm_241 0.1413 ms 99.3% ACC_TYPE='tl.float32', ALLOW_TF32=True, BLOCK_K=16, BLOCK_M=32, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=True, GROUP_M=8, num_stages=5, num_warps=8
  triton_mm_244 0.1413 ms 99.3% ACC_TYPE='tl.float32', ALLOW_TF32=True, BLOCK_K=16, BLOCK_M=64, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=True, GROUP_M=8, num_stages=2, num_warps=4
  triton_mm_245 0.1413 ms 99.3% ACC_TYPE='tl.float32', ALLOW_TF32=True, BLOCK_K=16, BLOCK_M=64, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=True, GROUP_M=8, num_stages=3, num_warps=8
  triton_mm_246 0.1413 ms 99.3% ACC_TYPE='tl.float32', ALLOW_TF32=True, BLOCK_K=16, BLOCK_M=64, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=True, GROUP_M=8, num_stages=5, num_

KeyboardInterrupt: 

In [ ]:
label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
true_labels, pred_labels = predict_on_test_set(model, test_loader, device, label_map)
test_acc = accuracy_score(true_labels, pred_labels)
print(f"\nTest Accuracy: {test_acc:.4f}")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(true_labels, pred_labels)

# Plot
class_names = list(label_map.values())
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# Compute TP, TN, FP, FN (only useful for binary, so here we show per class)
print("\nDetailed per-class statistics:")
for i, label in enumerate(class_names):
    TP = cm[i, i]
    FP = cm[:, i].sum() - TP
    FN = cm[i, :].sum() - TP
    TN = cm.sum() - (TP + FP + FN)
    print(f"\nClass '{label}':")
    print(f"  TP: {TP}")
    print(f"  TN: {TN}")
    print(f"  FP: {FP}")
    print(f"  FN: {FN}")

In [ ]:
plt.plot(history["train_loss"], label="Train Loss")
plt.plot(history["val_loss"], label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Train vs Val Loss")
plt.legend()
plt.show()